In [ ]:
# Imports
%%capture

!pip install datasets # !pip install datasets==1.0.2
!pip install transformers # !pip install transformers==4.0.1

import datasets
import pandas as pd

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import EncoderDecoderModel, BertTokenizerFast

In [ ]:
# Load data
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")

train_data.info.description
df = pd.DataFrame(train_data[:1])
del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


In [ ]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [ ]:
# Analyze data
def map_to_length(x):
    x["article_len"] = len(tokenizer(x["article"]).input_ids)
    x["article_longer_512"] = int(x["article_len"] > 512) # TODO: Set tokenizer.max_len
    x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
    x["summary_longer_64"] = int(x["summary_len"] > 64)
    x["summary_longer_128"] = int(x["summary_len"] > 128)
    
    return x
    
sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

def compute_and_print_stats(x):
    if len(x["article_len"]) == sample_size:
        print("Article Mean: {}\n%-Articles > 512: {}\nSummary Mean: {}\n%-Summary > 64: {}\n%-Summary > 128: {}\n".format(
                sum(x["article_len"]) / sample_size,
                sum(x["article_longer_512"]) / sample_size, 
                sum(x["summary_len"]) / sample_size,
                sum(x["summary_longer_64"]) / sample_size,
                sum(x["summary_longer_128"]) / sample_size)
        )

output = data_stats.map(
    compute_and_print_stats, 
    batched=True,
    batch_size=-1,
)

Article Mean: 869.4203
%-Articles > 512: 0.7478
Summary Mean: 59.9745
%-Summary > 64: 0.3829
%-Summary > 128: 0.0




In [ ]:
# Prepare data
encoder_max_length=512
decoder_max_length=128
batch_size = 4 # TODO: Set to 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

train_data = train_data.select(range(10000)) # TODO: Update limit
val_data = val_data.select(range(500))

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"],
)

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"],
)

In [ ]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=True)
tf2tf.save_pretrained("bert2bert_multilingual_shared_tied")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert_multilingual_shared_tied")

# print(tf2tf)
# print(tf2tf.config)
# print(tf2tf.num_parameters())

In [ ]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

In [ ]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [ ]:
# Imports
%%capture

!rm seq2seq_trainer.py
!rm seq2seq_training_args.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py

!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments

In [ ]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# Setup trainer
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch", # steps
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    warmup_steps=1000, # 2000
    num_train_epochs=3, # 50000
    save_steps=2000,
    logging_steps=2000,
    eval_steps=200, # 2000
    save_total_limit=3
)

# TODO: https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments
tf2tf = EncoderDecoderModel.from_pretrained("./checkpoint-6000")

trainer = Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
1,2.942600,6.412587,0.039800,0.049300,0.042700,394.638100,1.267000
2,2.492900,6.363551,0.049500,0.063000,0.053500,388.890600,1.286000
3,2.435100,6.099953,0.049700,0.061600,0.053000,401.797300,1.244000


TrainOutput(global_step=7500, training_loss=2.699346744791667, metrics={'train_runtime': 4795.1208, 'train_samples_per_second': 1.564, 'total_flos': 23838562137600000, 'epoch': 3.0})

In [ ]:
# Download latest checkpoint
import os
import numpy as np

from google.colab import files

directories = os.listdir("/content/")
checkpoints = [dir for dir in directories if "checkpoint" in dir]

values = [int(cp.split("-")[1]) for cp in checkpoints]
max_value = np.max(values)

path_zip = "/content/file.zip"
path_cp = "/content/checkpoint-" + str(max_value)

!zip -r "$path_zip" "$path_cp"
files.download(path_zip)
!rm -rf "$path_zip"

IndexError: ignored

In [ ]:
# Evaluate training
from transformers import BertTokenizer, BertTokenizerFast
tf2tf = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article"]
)

print(results[0]["pred_summary"])
print(results[0]["highlights"])
print("====================")

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid


best known for his portrayal of sheriff rosco p. coltrane on tv's " the dukes of hazzard " best died monday after a brief illness. best's co - stars paid tribute to the late actor on social media. " jimmy best was the most creative person i have ever known, " a friend says.
James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 .
"Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV .


Score(precision=0.10367853385391151, recall=0.15979827231902838, fmeasure=0.12265991541281784)